In [1]:
# This code uses predictions from individual models saved in files
# input files:
# CNN predictions in form of probabilities as .txt file: probability\tquestion, where probability - a prob. that a question is comp.
# BERT predictions in form of probabilities as .txt file: probability1\t\tprobaility2\tquestion, where probability1 is a prob. that a question is not comp., probability2 - otherwise
# ML (SVM and log. regr) predictions as .txt file: svm_probabilities\tlr_probabilities\tquestion

### Define helper functions

In [2]:
import glob
import os
import pandas as pd

def binary_pred(prob, thre=0.5):
    if prob >= thre: pred = 1
    else: pred = 0
    return pred

In [ ]:
dataset = 'train-multiclass-ya.txt' # annotated multicalss dataset
data = pd.read_csv('path_to_multiclass_dataset' + dataset, sep='\t')

In [ ]:
# Read BERT probibility predicitions for each subclass

classes = [ 'opinion/argument', 'reason/factoid', 'context/aspect', 'preference', 'direct', 'super', 'method']

i=0
for cl in classes:
    for n in range(10):     # considered 10 splits for 10-Fold Crossvalidation
        path_bert = 'path_to_BERT_multiclass_predictions'
        filename = glob.glob(os.path.join(path_bert, 'file' + classes[i] + str(n)))
        data_frame = pd.read_csv(filename[0], delimiter='\t', header=None)
        data_frame_ = pd.DataFrame({cl: data_frame[1]})
        if n == 0: data_frame_bert_ = data_frame_
        else: data_frame_bert_ = pd.concat([data_frame_bert_, data_frame_])
    if i == 0: data_frame_bert = data_frame_bert_  
    else: data_frame_bert[cl] = pd.Series(data_frame_bert_[cl].tolist(), index=data_frame_bert.index) 
    i+=1
    
# Read CNN probibility predicitions for into dataframe each crossvalidation split contains all subclasses in one file

for n in range(10):
    path_cnn = 'path_to_CNN_multiclass_predictions'
    filename = glob.glob(os.path.join(path_cnn, 'file' + str(n) + '.txt'))
    data_frame_ = pd.read_csv(filename[0], delimiter='\t')
    if n == 0: data_frame = data_frame_
    else: data_frame = pd.concat([data_frame, data_frame_])

In [ ]:
%%time

from sklearn.metrics import precision_recall_fscore_support, f1_score, recall_score, precision_score

# Calculate predictions for each subclass based on the probabilitites

classes = [ 'opinion/argument', 'reason/factoid', 'context/aspect', 'preference', 'direct', 'super', 'method']

precision_cnn = []
recall_cnn = []
f1_cnn = []

precision_bert = []
recall_bert = []
f1_bert = []

for cl in classes:
    if cl == 'preference': labels = [1.0 if x==2.0 else x for x in data[cl].tolist()] # convert labels 1 and 2 for preference to 1
    else: labels = data[cl].tolist()
        
    precision_cnn_cl = []
    recall_cnn_cl = []
    f1_cnn_cl = []
    precision_bert_cl = []
    recall_bert_cl = []
    f1_bert_cl = []
    
    for pr_pos in np.arange(0., 1.001, 0.0005):
        thre = round(pr_pos, 5)
        pred_cnn_ = []
        precision_cnn_ = []
        recall_cnn_ = []
        f1_cnn_ = []
        pred_bert_ = []
        precision_bert_ = []
        recall_bert_ = []
        f1_bert_ = []

        pred_cnn_ = [binary_pred(prob, thre) for prob in data_frame[cl].tolist()]
        pred_bert_ = [binary_pred(prob, thre) for prob in data_frame_bert[cl].tolist()]
        
        precision_cnn_ = round(precision_score(y_true=labels, y_pred=pred_cnn_, average='binary'),2)
        recall_cnn_ = round(recall_score(y_true=labels, y_pred=pred_cnn_, average='binary'),2)
        f1_cnn_ =  round(f1_score(y_true=labels, y_pred=pred_cnn_, average='binary'),2)
        
        precision_bert_ = round(precision_score(y_true=labels, y_pred=pred_bert_, average='binary'),2)
        recall_bert_ = round(recall_score(y_true=labels, y_pred=pred_bert_, average='binary'),2)
        f1_bert_ = round(f1_score(y_true=labels, y_pred=pred_bert_, average='binary'),2)
        
        precision_cnn_cl.append(precision_cnn_)
        recall_cnn_cl.append(recall_cnn_)
        f1_cnn_cl.append(f1_cnn_)
        
        precision_bert_cl.append(precision_bert_)
        recall_bert_cl.append(recall_bert_)
        f1_bert_cl.append(f1_bert_)

    precision_cnn.append(precision_cnn_cl)
    recall_cnn.append(recall_cnn_cl)
    f1_cnn.append(f1_cnn_cl)
    
    precision_bert.append(precision_bert_cl)
    recall_bert.append(recall_bert_cl)
    f1_bert.append(f1_bert_cl)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

classes = [ 'opinion/argument', 'reason/factoid', 'context/aspect', 'preference', 'direct', 'super', 'method']
titles = [ 'Opinion/argument', 'Reason/factoid', 'Context/aspect & Method', 'Preference', 'Direct', 'Super']

fig, axs = plt.subplots(2,3, figsize=(23, 9))
fig.subplots_adjust(hspace = .4, wspace=.001)
font = {'size' : 17}
plt.rc('font', **font)

major_ticks = np.arange(0, 1.01, 0.1)
minor_ticks = np.arange(0, 1.01, 0.05)

axs = axs.ravel()

for i in range(6):
    if i == 2:
        axs[2].plot(recall_bert[2], 
                    precision_bert[2], 
                    marker='o', label='BERT (context/aspect)', linestyle='dashed', 
                    linewidth=2, markersize=10, markevery = 0.05)

        axs[2].plot(recall_cnn[2],
                    precision_cnn[2], 
                    marker='d', label='CNN (context/aspect)', linestyle='dashed', 
                    linewidth=2, markersize=10, markevery = 0.05)
        
        axs[2].plot(recall_bert[6], 
                    precision_bert[6], 
                    marker='o', label='BERT (method)', linestyle='dashed', 
                    linewidth=2, markersize=10, markevery = 0.05)

        axs[2].plot(recall_cnn[6],
                    precision_cnn[6], 
                    marker='d', label='CNN (method)', linestyle='dashed', 
                    linewidth=2, markersize=10, markevery = 0.05)
        
    else:
        axs[i].plot(recall_bert[i], 
                    precision_bert[i], 
                    marker='o', label='BERT', linestyle='dashed', 
                    linewidth=2, markersize=10, markevery = 0.05)

        axs[i].plot(recall_cnn[i],
                    precision_cnn[i], 
                    marker='d', label='CNN', linestyle='dashed', 
                    linewidth=2, markersize=10, markevery = 0.05)
    
    
    axs[i].set_xticks(major_ticks)
    axs[i].set_xticks(minor_ticks, minor=True)
    axs[i].set_yticks(major_ticks)
    axs[i].set_yticks(minor_ticks, minor=True)
    
    axs[i].grid(which='minor', alpha=0.2)
    axs[i].grid(which='major', alpha=0.5)
    axs[i].grid(which='minor', alpha=0.2)
    axs[i].grid(which='major', alpha=0.5)
    
    axs[i].set_xlim([0.41,1.05])
    axs[i].set_ylim([0.66,1.02])
    axs[i].tick_params(axis='both', which='major', labelsize=16)
    
    axs[i].set_title(titles[i])
    axs[3].set_xlabel('Recall', fontsize = 17)
    axs[4].set_xlabel('Recall', fontsize = 17)
    axs[5].set_xlabel('Recall', fontsize = 17)
    axs[0].set_ylabel('Precision', fontsize = 17)
    axs[3].set_ylabel('Precision', fontsize = 17)

    axs[i].legend(loc=3, ncol=1)
    axs[i].legend(loc=3, ncol=1)
    axs[2].legend(loc=1, ncol=1)
    axs[1].legend(loc=3, ncol=1)
    
plt.show()